<a href="https://colab.research.google.com/github/kayla-jackson/spatial-modeling/blob/test-exploratory-sims/notebooks/spatially_correlated_p.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [21]:
# Helper functions to generate a sample from MVN(0, sigma)

def adjacency_matrix(A):
    '''Returns the adjacency matrix W from A, a NumPy array'''
    rows, cols = len(A), len(A[0])
    locs = rows * cols # number of spatial locations on the grid
    W = np.zeros((locs, locs), dtype=int)

    # Iterate through each element in A to determine number of neighbors
    for r in range(rows):
        for c in range(cols):
            for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
                if 0 <= r + dr < rows and 0 <= c + dc < cols:
                    W[r * cols + c][(r + dr) * cols + c + dc] = 1

    return W

def sum_adjacency_matrix(W):
    '''Outputs matrix D with same dimensions as W;
    Sums the number of neighbors of the original points'''
    return np.diag(np.sum(W, axis=1))

def compute_sigma(A, alpha, tau):
    '''Outputs the covariance matrix termed sigma from original matrix A'''
    W = adjacency_matrix(A)
    D = sum_adjacency_matrix(W)

    Q = tau * (D - alpha * W) # the formula
    return np.linalg.inv(Q)

In [22]:
# Testing the functions

A = np.array([[1,2,3],[4,5,6],[7,8,9]])
W = adjacency_matrix(A)
D = sum_adjacency_matrix(W)
sigma = compute_sigma(A, 0.99, 1)

print(f'W: {W}')
print(f'D: {D}')
print(f'Sigma: {sigma}')

W: [[0 1 0 1 0 0 0 0 0]
 [1 0 1 0 1 0 0 0 0]
 [0 1 0 0 0 1 0 0 0]
 [1 0 0 0 1 0 1 0 0]
 [0 1 0 1 0 1 0 1 0]
 [0 0 1 0 1 0 0 0 1]
 [0 0 0 1 0 0 0 1 0]
 [0 0 0 0 1 0 1 0 1]
 [0 0 0 0 0 1 0 1 0]]
D: [[2 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0]
 [0 0 0 0 4 0 0 0 0]
 [0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 3 0]
 [0 0 0 0 0 0 0 0 2]]
Sigma: [[4.72557687 4.26825946 4.10427136 4.26825946 4.10427136 4.02319782
  4.10427136 4.02319782 3.98296585]
 [4.26825946 4.51847503 4.26825946 4.10427136 4.14572864 4.10427136
  4.02319782 4.02340102 4.02319782]
 [4.10427136 4.26825946 4.72557687 4.02319782 4.10427136 4.26825946
  3.98296585 4.02319782 4.10427136]
 [4.26825946 4.10427136 4.02319782 4.51847503 4.14572864 4.02340102
  4.26825946 4.10427136 4.02319782]
 [4.10427136 4.14572864 4.10427136 4.14572864 4.35427136 4.14572864
  4.10427136 4.14572864 4.10427136]
 [4.02319782 4.10427136 4.26825946 4.02340102 4.14572864 4.51847503
  4.02319782 4.1042

In [23]:
# With the full 10x10 grid:

A = np.zeros((10,10), dtype=int)
W = adjacency_matrix(A)
D = sum_adjacency_matrix(W)
sigma = compute_sigma(A, 0.99, 1)

print(f'W: {W}')
print(f'D: {D}')
print(f'Sigma: {sigma}')

W: [[0 1 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 1]
 [0 0 0 ... 0 1 0]]
D: [[2 0 0 ... 0 0 0]
 [0 3 0 ... 0 0 0]
 [0 0 3 ... 0 0 0]
 ...
 [0 0 0 ... 3 0 0]
 [0 0 0 ... 0 3 0]
 [0 0 0 ... 0 0 2]]
Sigma: [[1.44637106 0.95593036 0.68112832 ... 0.14572908 0.1416529  0.14023637]
 [0.95593036 1.16191315 0.77788798 ... 0.14658889 0.14288171 0.1416529 ]
 [0.68112832 0.77788798 1.04330611 ... 0.14958705 0.14658889 0.14572908]
 ...
 [0.14572908 0.14658889 0.14958705 ... 1.04330611 0.77788798 0.68112832]
 [0.1416529  0.14288171 0.14658889 ... 0.77788798 1.16191315 0.95593036]
 [0.14023637 0.1416529  0.14572908 ... 0.68112832 0.95593036 1.44637106]]


In [35]:
# Generating the sample

def MVN_sample(sigma):
    '''Returns a sample from a multivariate normal distribution
    where sigma is a covariance matrix'''
    zero_vector = np.zeros(len(sigma))
    sample = np.random.multivariate_normal(zero_vector, sigma)

    return sample

def inverse_logit_transformation(M):
    '''Transforms a matrix of values into the range (0,1)'''
    return 1 / (1 + np.exp(-M))

In [36]:
# Testing the above functions

sample = MVN_sample(sigma)
transformed_sample = inverse_logit_transformation(sample)
print(sample)
print(transformed_sample)

[ 0.09936512 -0.45510788  0.60860486  0.52885871  0.1107455   0.64851774
  1.12686319  0.04759283 -0.41708648 -1.31719387  0.11479255 -0.33338361
  0.13125119  0.75964191 -0.31166391  0.70693891 -0.59279287 -1.48563689
 -1.25307457 -0.94267524 -0.22674093 -0.45763101  0.39959344  1.00972887
  0.93708813  0.1429594  -1.09992156 -0.39082006 -0.52125283 -0.29004109
  0.14451038  0.41003379  0.97221333  0.1685332  -0.17630832  0.24777113
 -0.3736659  -1.45983176 -0.13227807  0.29427531 -0.14127138  0.57899608
  0.07487003 -0.16211306  0.04111261 -0.92263054 -1.07517867 -0.54702808
 -0.09093499 -0.20273163  0.5549732  -0.73730067  0.40239985 -0.8071676
 -0.98080733 -1.00325829 -1.34542026 -0.21451476 -0.36817926 -0.31305604
 -1.53702238 -1.10706804 -0.8639632  -0.86516002 -0.03032462  0.2139956
 -0.2752821  -0.4487904  -0.33842837  0.45151585 -1.16012148 -0.13060761
 -0.76698662 -1.24452408 -0.49779353  0.5192837   1.05261978  0.95815927
  0.02331258  0.88254307 -0.88856868 -0.32020292 -0.9